# Julia and Vectorization

## 1. What is Vectorization?

**Vectorization** is the art of getting rid of explicit ```for``` loops in code.

In the deep learning era, you often find yourself training on relatively large datasets, because that is when deep learning algorithms tend to shine. So, it is important that your code runs very quickly, and the ability to perform vectorization has become a key skill.

In logistic regression, we have 

$$
z = w^T x + b,
$$

where $w, x \in \mathbb{R}^{n_x}$.

To computec $z$, a non vectorized way is:

```julia
z = 0
for i in 1: n_x:
    z += w[i] * x[i]
z = z .+ b
```

which can be very slow. In contrast, the vectorized version

```julia
using LinearAlgebra

z = dot(w, x) .+ b
```

is much faster.

In [1]:
using Random
using LinearAlgebra
using BenchmarkTools

len = 1000000
rng1 = MersenneTwister(1234)
a = rand(rng1, len)
rng2 = MersenneTwister(12345)
b = rand(rng2, len)


function inner_product(a, b)
    result = 0
    for i in 1: length(a) 
        result += a[i] * b[i] 
    end
    return result
end


@btime dot($a, $b)
@btime inner_product($a, $b);

  297.942 μs (0 allocations: 0 bytes)
  956.823 μs (0 allocations: 0 bytes)


## 2. Does Julia Need Vectorization?

Vectorization is used to speed up the code without using loop. Using such a function can help in minimizing the running time of code efficiently. There are two meanings of the word vectorization in the high-level languages, and they refer to different things.
When we talk about vectorized code in Python/Numpy/MATLAB etc., we are usually referring to the fact that code be like:

```python
x = [1, 2, 3] 
y = x + 1
```

is faster than:

```python

x = [1, 2, 3] 
for i in 1:3  
  y[i] = x[i] + 1
end 
```

The kind of vectorization in the first code block is quite helpful in languages like Python and MATLAB because *generally, every operation in these languages tends to be slow*. Each iteration involves calling ```+``` operator, making array lookups, type-conversions etc. and repeating this iteration for a given number of times makes the overall computation slow. So, it's faster to vectorize the code and only paying the cost of looking up the ```+``` operation once for the entire vector ```x``` rather than once for each element ```x[i]```.

We don't come across such problems in Julia, where

```julia
y .= x .+ 1
```

and 

```julia
for i in 1:3
    y[i] = x[i] + 1
end 
```

both compile down to almost the same code and perform comparably. So the kind of vectorization needed in Python and MATLAB is not necessary in Julia.

In [2]:
len = 1000000
rng = MersenneTwister(1234)
a = rand(rng, len)


function vectorized(a)
    b .= a .+ 1
    return b
end 

b = zeros(length(a))
function nonvectorized!(a, b)
    for i in 1: length(a)
        b[i] = a[i] + 1
    end
    return b
end


@btime vectorized($a)
@btime nonvectorized!($a, $b);

  355.558 μs (1 allocation: 32 bytes)
  499.476 μs (0 allocations: 0 bytes)


As we can see, although vectorization in this case is actually faster than non-vectorization, it is not that advantageous as in the Python case.

In fact, *each vectorized operation ends up generating a new temporary array and executing a separate loop, which leads to a lot of overhead when multiple vectorized operations are combined.* So in some cases, we may observe vectorized code to run slower.

The other kind of vectorization pertains to improving performance by using **SIMD (Single Instruction, Multiple Data)** instructions and refers to the CPU's ability to operate on chunks of data.

### 2.1 Single Instruction, Multiple Data(SIMD)

The following code shows a simple sum function (returning the sum of all the elements in an array ```arr```):

In [3]:
function mysum(arr::Vector) 
    total = zero(eltype(arr)) 
    for x in arr 
        total += x 
    end 
    return total 
end 

mysum (generic function with 1 method)

This operation can be generally visualized as serial addition of ```total``` with an array element in every iteration.

In [4]:
# generate random data 
rng = MersenneTwister(123)
arr = rand(rng, 10^5)
@benchmark mysum($arr)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     83.535 μs (0.00% GC)
  median time:      108.406 μs (0.00% GC)
  mean time:        117.555 μs (0.00% GC)
  maximum time:     476.823 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

Using ```@simd``` can speed up the ```for``` loop.

In [5]:
function mysum1(arr::Vector) 
    total = zero(eltype(arr)) 
    @simd for x in arr 
        total += x 
    end 
    return total 
end 
  
# benchmark the mysum1 function 
@benchmark mysum1($arr) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     11.240 μs (0.00% GC)
  median time:      14.703 μs (0.00% GC)
  mean time:        17.539 μs (0.00% GC)
  maximum time:     143.648 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

We can clearly see the performance increase after using the SIMD macro. So what's actually happening ? By taking advantage of the SIMD instruction set (inbuilt feature of most Intel® CPU's), the add operation is performed in two steps:

1. During the first step, intermediate values are accumulated $n$ at a time ($n$ depends on the CPU hardware).
2. In a so called **reduction step**, the final $n$ elements are summed together.

Now the question arises, how can we combine SIMD vectorization and the language's vectorization capabilities to derive more performance both from the language's compiler and the CPU? In Julia we answer it with the ```LoopVectorization.jl``` package.

### 2.2 Loop Vectorization

**```LoopVectorization.jl``` is a Julia package that provides macros and functions that combine SIMD vectorization and loop reordering so as to improve performance.**

```julia
@avx macro
```

It annotates a ```for``` loop, or a set of nested ```for``` loops whose *bounds are constant* across iterations, to optimize the computation.

Let's consider the classical dot product problem. To know more about dot product and it's vectorized implementation in python, check out this [article](https://www.geeksforgeeks.org/vectorization-in-python/). In the below examples we will benchmark the same code with different types of vectorization.

In [6]:
# Without using any macro 
function dotProd(x, y) 
    prod = zero(eltype(x)) 
    for i in eachindex(x, y) 
        prod += x[i] * y[i] 
    end 
    prod 
end 

# using the simd macro 
function dotProd_simd(x, y) 
    prod = zero(eltype(x)) 
    @simd for i in eachindex(x, y) 
        prod += x[i] * y[i] 
    end 
    prod 
end 

# using the avx macro 
using LoopVectorization 
function dotProd_avx(x, y) 
    prod = zero(eltype(x)) 
    @avx for i in eachindex(x, y) 
        prod += x[i] * y[i] 
    end 
    prod 
end 


dotProd_avx (generic function with 1 method)

In [7]:
# generating random data 
rng = MersenneTwister(123)
x = rand(rng, 10^5); 
y = rand(rng, 10^5); 
  
# benchmark the function without any macro 
@benchmark dotProd($x, $y)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     80.212 μs (0.00% GC)
  median time:      90.004 μs (0.00% GC)
  mean time:        104.487 μs (0.00% GC)
  maximum time:     457.943 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [8]:
# benchmark the function with simd macro 
@benchmark dotProd_simd($x, $y)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     83.556 μs (0.00% GC)
  median time:      106.612 μs (0.00% GC)
  mean time:        133.663 μs (0.00% GC)
  maximum time:     1.063 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [9]:
# benchmark the function with avx macro 
@benchmark dotProd_avx($x, $y) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     21.046 μs (0.00% GC)
  median time:      27.362 μs (0.00% GC)
  mean time:        30.701 μs (0.00% GC)
  maximum time:     262.207 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

We observe that the ```@avx``` macro turns out to have the best performance! The time gap will generally increase for larger sizes of $x$ and $y$.

## 3. Julia versus Python

Julia is a *compiled* language, while Python is an *interpreted* language. In some cases Python interpreter can provide fast execution speed, but in other cases it currently is not able to do it (but it does not mean that in the future it will not improve). In particular vectorized functions (like ```dot``` in Julia or ```np.dot``` in Python) are most likely written in some compiled language, so Julia and Python will not differ much in typical cases as they just call this *compiled* function. *However, when you use loops (non-vectorized code) then currently Python will be slower than Julia.* In Julia there is no performance penalty, as long as you write type stable code and use ```a@vx``` (or ```@simd``` and ```@inbounds``` instead) where required. 